In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
f = files.upload()

Saving sherloch holmes.txt to sherloch holmes.txt


In [3]:
f = open("sherloch holmes.txt", "r", encoding = "utf8")
lines = []
for i in f:
    lines.append(i)

data = ""
for i in lines:
  data = ' '. join(lines) 

data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

data = data.split()
data = ' '.join(data)

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[142, 4680, 1, 986, 5, 125, 33, 46, 556, 2164, 2165, 27, 987, 14, 22]

In [6]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

8624


In [7]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-4:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  108955


<ipython-input-7-c43534fdb19a>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences = np.array(sequences)


array([list([]), list([142, 4680, 1, 986, 5]),
       list([4680, 1, 986, 5, 125]), list([1, 986, 5, 125, 33]),
       list([986, 5, 125, 33, 46]), list([5, 125, 33, 46, 556]),
       list([125, 33, 46, 556, 2164]), list([33, 46, 556, 2164, 2165]),
       list([46, 556, 2164, 2165, 27]), list([556, 2164, 2165, 27, 987])],
      dtype=object)

In [8]:
X = []
y = []

for i in sequences:
    if len(i) < 4:
      continue
    X.append(i[0:4])
    y.append(i[-1])
    
X = np.array(X)
y = np.array(y)

In [9]:
X[0]

array([ 142, 4680,    1,  986])

In [10]:
y = to_categorical(y, num_classes=vocab_size)
y[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [11]:
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.models import Sequential

In [12]:
import math
from keras import backend as K

def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perplexity = K.pow(2.0, cross_entropy)
    return perplexity

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=len(X[0])))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 10)             86240     
                                                                 
 lstm (LSTM)                 (None, 4, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 8624)              8632624   
                                                                 
Total params: 21,767,864
Trainable params: 21,767,864
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=[perplexity])
model.fit(X, y, epochs=30, batch_size= 512)

Epoch 1/30
213/213 [==============================] - 34s 112ms/step - loss: 6.7696 - perplexity: 606.7615
Epoch 2/30
213/213 [==============================] - 16s 76ms/step - loss: 6.3853 - perplexity: 434.6445
Epoch 3/30
213/213 [==============================] - 13s 63ms/step - loss: 6.0865 - perplexity: 434.1695
Epoch 4/30
213/213 [==============================] - 13s 63ms/step - loss: 5.8215 - perplexity: 329.1391
Epoch 5/30
213/213 [==============================] - 14s 63ms/step - loss: 5.5543 - perplexity: 270.0941
Epoch 6/30
213/213 [==============================] - 14s 66ms/step - loss: 5.3032 - perplexity: 182.4940
Epoch 7/30
213/213 [==============================] - 14s 65ms/step - loss: 5.0683 - perplexity: 137.4905
Epoch 8/30
213/213 [==============================] - 14s 65ms/step - loss: 4.8285 - perplexity: 102.4115
Epoch 9/30
213/213 [==============================] - 14s 68ms/step - loss: 4.5759 - perplexity: 7656.4272
Epoch 10/30
213/213 [=======================

In [16]:
model.save('sherlock_holmes.h5')

In [17]:
def pred(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds = np.argmax(model.predict(sequence, verbose = 0))
    predicted_word = ""

    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
    
    return predicted_word

In [18]:
while(True):
  text = input("\n\nEnter your line: ")
  if text == "0":
      print("Execution completed.....")
      break  
  else:
      try:
          ans = ''
          for i in range(15): 
            predicted_word = pred(model, tokenizer, text.split()[-4:])
            # if stopwords(predicted_word):
            #   print("Reached End Of Sentence")
            #   break
            ans += predicted_word+' '
            text += ' '+ predicted_word
          print('\t' + ans)
      except Exception as e:
        print("Error occurred: ",e)
        continue



Enter your line: this may save us a visit
	to brixton road whispered holmes come with me and we will see what is to 


Enter your line: Whatever were you doing with
	that bird jem ’ says she ‘well ’ said i ‘you may absolutely depend upon 


Enter your line: Life is infinitely stranger than
	anything which the mind of man could invent we would not dare to conceive the 


Enter your line: 0
Execution completed.....
